In [55]:
import requests
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq

In [58]:
import requests
import pandas as pd

# Set the base URL for the CDC data
base_url = "https://data.cdc.gov/resource/9bhg-hcku.json"

# Initialize parameters for pagination
limit = 50000  # Maximum number of results per request (can be set up to 50000 for Socrata APIs)
offset = 0    # Start at the beginning
data = []

# Loop until no more data is returned
while True:
    # Update the URL with the current offset
    data_url = f"{base_url}?$limit={limit}&$offset={offset}"
    
    # Make the API request
    response = requests.get(data_url)
    page_data = response.json()
    
    # If no more data is returned, break the loop
    if not page_data:
        break
    
    # Append the results and update the offset
    data.extend(page_data)
    offset += limit

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Ensure the datetime is in UTC and format it for BigQuery
df['data_as_of'] = pd.to_datetime(df['data_as_of'])
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

# Convert columns that should be integers from string to int
columns_to_convert = [
    'covid_19_deaths', 
    'total_deaths', 
    'pneumonia_deaths', 
    'influenza_deaths',
    'pneumonia_and_covid_19_deaths', 
    'pneumonia_influenza_or_covid', 
    'year', 
    'month', 
    # add other column names here that need to be converted
]

for column in columns_to_convert:
    if column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

# Now df contains more than 1000 rows, if available
df.head()


,data_as_of,start_date,end_date,group,state,sex,age_group,covid_19_deaths,total_deaths,pneumonia_deaths,pneumonia_and_covid_19_deaths,influenza_deaths,pneumonia_influenza_or_covid,footnote,year,month
0,2023-09-27,2020-01-01,2023-09-23,By Total,United States,All Sexes,All Ages,1146774,12303399,1162844,569264,22229,1760095,NaN,<NA>,<NA>
1,2023-09-27,2020-01-01,2023-09-23,By Total,United States,All Sexes,Under 1 year,519,73213,1056,95,64,1541,NaN,<NA>,<NA>
2,2023-09-27,2020-01-01,2023-09-23,By Total,United States,All Sexes,0-17 years,1696,130970,2961,424,509,4716,NaN,<NA>,<NA>
3,2023-09-27,2020-01-01,2023-09-23,By Total,United States,All Sexes,1-4 years,285,14299,692,66,177,1079,NaN,<NA>,<NA>
4,2023-09-27,2020-01-01,2023-09-23,By Total,United States,All Sexes,5-14 years,509,22008,818,143,219,1390,NaN,<NA>,<NA>


from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Initialize the BigQuery client
credentials = service_account.Credentials.from_service_account_file(
    '/Users/naveenreddy/Documents/1st sem/4.Data Management/Final Project/airy-actor-405720-108ba356c83b.json'
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define your BigQuery dataset and table
dataset_id = 'Environmental'
table_id = 'AirQuality'

# Define the table schema
schema = [
    bigquery.SchemaField("measureid", "STRING"),
    bigquery.SchemaField("measurename", "STRING"),
    bigquery.SchemaField("measuretype", "STRING"),
    bigquery.SchemaField("stratificationlevel", "STRING"),
    bigquery.SchemaField("statefips", "STRING"),
    bigquery.SchemaField("statename", "STRING"),
    bigquery.SchemaField("countyfips", "STRING"),
    bigquery.SchemaField("countyname", "STRING"),
    bigquery.SchemaField("reportyear", "STRING"),
    bigquery.SchemaField("value", "STRING"),  # Assuming it can be float
    bigquery.SchemaField("unit", "STRING"),
    bigquery.SchemaField("unitname", "STRING"),
    bigquery.SchemaField("dataorigin", "STRING"),
    bigquery.SchemaField("monitoronly", "STRING"),
]

# Create the dataset in BigQuery if it doesn't exist
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)

# Create the table in BigQuery if it doesn't exist
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref, schema=schema)
client.create_table(table, exists_ok=True)

# Load your data into a Pandas DataFrame (replace this with your actual data loading code)
# Assuming you have a DataFrame named 'df' containing your data
# df = pd.read_csv('your_data.csv')

# Insert the data into the BigQuery table with the specified schema
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

# Check for errors in the job
if job.errors:
    print("Job completed with errors:", job.errors)
else:
    print("Data inserted successfully.")


In [57]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Initialize the BigQuery client
credentials = service_account.Credentials.from_service_account_file(
    '/Users/naveenreddy/Documents/1st sem/4.Data Management/Final Project/airy-actor-405720-108ba356c83b.json'
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define your BigQuery dataset and table
dataset_id = 'Carona'
table_id = 'Death'

# Define the table schema
schema = [
    bigquery.SchemaField("data_as_of", "TIMESTAMP"),
    bigquery.SchemaField("start_date", "TIMESTAMP"),
    bigquery.SchemaField("end_date", "TIMESTAMP"),
    bigquery.SchemaField("group", "STRING"),
    bigquery.SchemaField("state", "STRING"),
    bigquery.SchemaField("sex", "STRING"),
    bigquery.SchemaField("age_group", "STRING"),
    bigquery.SchemaField("covid_19_deaths", "INTEGER"),
    bigquery.SchemaField("total_deaths", "INTEGER"),
    bigquery.SchemaField("pneumonia_deaths", "INTEGER"),
    bigquery.SchemaField("pneumonia_and_covid_19_deaths", "INTEGER"),
    bigquery.SchemaField("influenza_deaths", "INTEGER"),
    bigquery.SchemaField("pneumonia_influenza_or_covid", "INTEGER"),
    bigquery.SchemaField("footnote", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
]

# Create the dataset in BigQuery if it doesn't exist
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)

# Create the table in BigQuery if it doesn't exist
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref, schema=schema)
client.create_table(table, exists_ok=True)

# Insert the data into the BigQuery table with the specified schema
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

# Check for errors in the job
if job.errors:
    print("Job completed with errors:", job.errors)
else:
    print("Data inserted successfully.")


Data inserted successfully.
